In [18]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import fitsio
from fitsio import FITS,FITSHDR
from astropy import units as u
from astropy.coordinates import SkyCoord
from PIL import Image
from scipy import stats
from astropy.table import Table, Column
from astropy.io import ascii

In [2]:
data = '/Users/amyel/research/SMASHproject/datafiles/matched_data.fits'
fx = fitsio.FITS(data)
objs = fx[1].read()

In [3]:
objs


array([( 14.415298  , 0.718604  , 5297.5   , 4.4272823, -0.6448489 , b'1237663204920459328', 19.036293, 17.480038, 16.890882 , 16.660769, 0.00835069, 0.00228338, 0.00159616, 0.02609584,  True),
       ( 14.519823  , 0.807096  , 4545.8696, 4.6285467, -0.95265365, b'1237663204920525012', 21.29038 , 19.181652, 18.204958 , 17.857706, 0.07730225, 0.00613446, 0.00415108, 0.02691421,  True),
       ( 14.5311    , 0.791671  , 6507.0327, 3.328773 , -2.9210906 , b'1237663204920525018', 19.732542, 18.915205, 18.68072  , 18.574429, 0.012401  , 0.00551958, 0.00430055, 0.02757653,  True),
       ...,
       (353.53997803, 0.230864  , 5773.    , 4.36     ,  0.08      , b'1237666408439611400', 17.092207, 15.664094, 15.182018 , 15.022112, 0.00838887, 0.00439602, 0.00282024, 0.03289959, False),
       (353.54449463, 0.28760201, 4567.    , 4.56     , -0.43      , b'1237666408439611402', 18.875969, 16.863127, 16.016212 , 15.698523, 0.01586362, 0.00357837, 0.00237041, 0.03166599, False),
       (352.759033

In [4]:
u_g_matched = (objs["U"]-(4.239*objs["EBV"]))-(objs["G"]-(3.303*objs["EBV"]))
g_r_matched = (objs["G"]-(3.303*objs["EBV"]))-(objs["R"]-(2.285*objs["EBV"]))

In [5]:
#using both SDSS and LAMOST data
ret,xedge,yedge,bin = stats.binned_statistic_2d(u_g_matched,g_r_matched,objs["FEH"],'median',50)

/Users/amyel/anaconda3/lib/python3.7/site-packages/scipy/stats/_binned_statistic.py:607: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = result[core]


In [33]:
#create empty lists to be used in metallicity funciton
#list of field number
field_number = []
#list of initial star count
initial_star_count = []
#list of star count after color/magnitude cut
cut_star_count = []
#list of mean metallicity
mean_metallicity = []
#list of median metallicity
median_metallicity = []
#list of standard deviation for each field
std = []

In [34]:
def metallicity(filename):
    
    #read fits file
    fx = fitsio.FITS(filename)
    objs_new = fx[1].read()
    
    #extract field number from filename string
    newstr = ''.join((ch if ch in '0123456789' else ' ') for ch in filename)
    field_number_transient = [int(i) for i in newstr.split()]
    print("number of initial stars in field "+str(field_number_transient[0])+": "+str(len(objs_new)))
    
    #append to lists
    field_number.append(field_number_transient[0])
    initial_star_count.append(len(objs_new))
    
    #deredden the colors from input file
    u = (objs_new["U"]-(4.239*objs_new["EBV"]))
    g = (objs_new["G"]-(3.303*objs_new["EBV"]))
    i = (objs_new["I"]-(1.263*objs_new["EBV"]))
    r = (objs_new["R"]-(2.285*objs_new["EBV"]))

    #make cuts
    selected = np.where(((g-i)>=0.11)&((g-i)<=0.44)&(g>=21.4)&(g<=22.8))
    cut_u = u[selected]
    cut_g = g[selected]
    cut_i = i[selected]
    cut_r = r[selected]
    u_g = cut_u-cut_g
    g_r = cut_g-cut_r
    g_i = cut_g-cut_r
    
    #append to list
    cut_star_count.append(len(cut_u))
    print("number of  stars in field "+str(field_number_transient[0])+" after color/magnitude cuts: "+str(len(cut_u)))
    
    #read out metallicity values using yumi's method
    ind, = np.where((u_g>=xedge.min())&(u_g<=xedge.max())&(g_r>=yedge.min())&(g_r<=yedge.max()))
    ix = np.searchsorted(xedge,u_g[ind])
    ix = ix-1
    iy = np.searchsorted(yedge,g_r[ind])
    iy = iy-1
    new_metal = ret[ix,iy]
    
    #calculate mean, median, standard devitation; print values; append appropriate lists 
    mean = np.nanmean(new_metal)
    median = np.nanmedian(new_metal)
    std_value = np.nanstd(new_metal)
    print("median: ",median)
    print("mean: ",mean)
    print("standard deviation: ",std_value)
    mean_metallicity.append(mean)
    median_metallicity.append(median)
    std.append(std_value)
    
    
    #NOTE: below: add offset of .7 after comparing to APOGEE spectroscopy
    #find medium, mean metallicity
    #mean = np.nanmean(new_metal)+.7
    #median = np.nanmedian(new_metal)+.7
    #print("median: ",median)
    #print("mean: ",mean)
    
    return 0

In [35]:
#SMASH field arrays
LMC_outer_fields = [156,24,246,26,27,28,29,31,33,44,52,53,54,55,56,57,58,59,60,61,63,64,66,68]
LMC_inner_fields = [30,32,34,35,37,40,42,46,48,49,50,51]
SMC_outer_fields = [1,13,149,150,176,177,178,18,20,21,22,8]
SMC_inner_fields = [12,14,15,16,19,2,3,4,5,7,9]



### Run metallicity function for LMC periphery fields

In [97]:
if __name__=="__main__":
    for number in LMC_outer_fields:
        metallicity('/Users/amyel/research/SMASHproject/datafiles/SMASH_fields/vsix/starsthree/Field{}_allobj_deep_stars.fits.gz'.format(number))

              

number of initial stars in field 156: 122018
number of  stars in field 156 after color/magnitude cuts: 1285
median:  -1.4411659240722656
mean:  -1.3765135254368794
standard deviation:  0.6549224927233428
number of initial stars in field 24: 29599
number of  stars in field 24 after color/magnitude cuts: 715
median:  -1.447492003440857
mean:  -1.4006474511749918
standard deviation:  0.6829247863499761
number of initial stars in field 246: 76714
number of  stars in field 246 after color/magnitude cuts: 1017
median:  -1.3419320583343506
mean:  -1.1354115158319473
standard deviation:  0.6667843377530529
number of initial stars in field 26: 52301
number of  stars in field 26 after color/magnitude cuts: 4596
median:  -1.4035937786102295
mean:  -1.3138622322813012
standard deviation:  0.6511373465144901
number of initial stars in field 27: 94878
number of  stars in field 27 after color/magnitude cuts: 21383
median:  -1.447492003440857
mean:  -1.385981988836408
standard deviation:  0.6889421855

In [98]:
print(field_number)
print(len(field_number))
print(initial_star_count)
print(len(initial_star_count))
print(cut_star_count)
print(len(cut_star_count))
print(mean_metallicity)
print(len(mean_metallicity))
print(median_metallicity)
print(len(median_metallicity))
print(std)
print(len(std))

[156, 24, 246, 26, 27, 28, 29, 31, 33, 44, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 66, 68]
24
[122018, 29599, 76714, 52301, 94878, 114705, 404098, 46943, 45996, 427557, 62890, 396092, 163332, 637616, 120917, 76423, 77341, 88849, 59324, 80665, 113977, 68421, 101460, 90704]
24
[1285, 715, 1017, 4596, 21383, 18916, 99393, 1703, 1933, 82065, 3185, 155432, 27333, 148292, 8537, 895, 1478, 3170, 1817, 2447, 2477, 1598, 1409, 1304]
24
[-1.3765135254368794, -1.4006474511749918, -1.1354115158319473, -1.3138622322813012, -1.385981988836408, -1.3197357342063933, -1.3604937254725722, -1.4210028691110101, -1.321077586850585, -1.670842062044637, -1.6257458581970772, -1.4108179055812022, -1.4440663630870465, -1.357551053241799, -1.8470588968920985, -1.5821966921361619, -1.3927393218669855, -1.3993819994904115, -1.6162108487513172, -1.5808480663949416, -1.7364739961791227, -1.5222834335913853, -1.4174050767741215, -1.3581208291221543]
24
[-1.4411659240722656, -1.447492003440857, -1.341932058334

In [99]:
print("Total number of initial LMC periphery stars before: "+str(sum(initial_star_count)))
print("Total number of LMC periphery stars after color/magnitude cuts: "+str(sum(cut_star_count)))

Total number of initial LMC periphery stars before: 3552820
Total number of LMC periphery stars after color/magnitude cuts: 592380


### Make table for 24 SMASH LMC periphery fields

In [100]:
arrays = [(np.array(field_number)),(np.array(initial_star_count)),(np.array(cut_star_count)),(np.array(mean_metallicity)).round(decimals=3),(np.array(median_metallicity)).round(decimals=3),(np.array(std)).round(decimals=3)]


In [74]:
outer_LMC_table = Table(arrays,names=('Field Number','Initial Star Count','Star Count after Color/Magnitude Cuts','Mean Metallicity','Median Metallicity','Standard Deviation'))



In [84]:

outer_LMC_table.pprint(max_lines=-1, max_width=-1)

Field Number Initial Star Count Star Count after Color/Magnitude Cuts Mean Metallicity Median Metallicity Standard Deviation
------------ ------------------ ------------------------------------- ---------------- ------------------ ------------------
         156             122018                                  1285           -1.377             -1.441              0.655
          24              29599                                   715           -1.401             -1.447              0.683
         246              76714                                  1017           -1.135             -1.342              0.667
          26              52301                                  4596           -1.314             -1.404              0.651
          27              94878                                 21383           -1.386             -1.447              0.689
          28             114705                                 18916            -1.32             -1.404              0.663


### Write table to file

In [80]:


ascii.write(outer_LMC_table,'24 SMASH LMC Periphery Fields.csv',format='csv', fast_writer=False)


In [83]:
import csv
with open('24 SMASH LMC Periphery Fields.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

['Field Number', 'Initial Star Count', 'Star Count after Color/Magnitude Cuts', 'Mean Metallicity', 'Median Metallicity', 'Standard Deviation']
['156', '122018', '1285', '-1.377', '-1.441', '0.655']
['24', '29599', '715', '-1.401', '-1.447', '0.683']
['246', '76714', '1017', '-1.135', '-1.342', '0.667']
['26', '52301', '4596', '-1.314', '-1.404', '0.651']
['27', '94878', '21383', '-1.386', '-1.447', '0.689']
['28', '114705', '18916', '-1.32', '-1.404', '0.663']
['29', '404098', '99393', '-1.36', '-1.441', '0.648']
['31', '46943', '1703', '-1.421', '-1.53', '0.653']
['33', '45996', '1933', '-1.321', '-1.404', '0.677']
['44', '427557', '82065', '-1.671', '-1.72', '0.57']
['52', '62890', '3185', '-1.626', '-1.72', '0.603']
['53', '396092', '155432', '-1.411', '-1.447', '0.65']
['54', '163332', '27333', '-1.444', '-1.53', '0.622']
['55', '637616', '148292', '-1.358', '-1.441', '0.653']
['56', '120917', '8537', '-1.847', '-1.91', '0.47']
['57', '76423', '895', '-1.582', '-1.665', '0.549']
[

### Run metallicity function for LMC main body fields

In [104]:
if __name__=="__main__":
    for number in LMC_inner_fields:
        metallicity('/Users/amyel/research/SMASHproject/datafiles/SMASH_fields/vsix/starsthree/Field{}_allobj_deep_stars.fits.gz'.format(number))

              

number of initial stars in field 30: 1551563
number of  stars in field 30 after color/magnitude cuts: 110478
median:  -1.274411916732788
mean:  -1.2790842923674013
standard deviation:  0.670759679741447
number of initial stars in field 32: 1825281
number of  stars in field 32 after color/magnitude cuts: 360148
median:  -1.1198506355285645
mean:  -1.1976510338658024
standard deviation:  0.6646917111479915
number of initial stars in field 34: 1421909
number of  stars in field 34 after color/magnitude cuts: 423023
median:  -1.447492003440857
mean:  -1.3951076149374833
standard deviation:  0.6380655075651893
number of initial stars in field 35: 1606580
number of  stars in field 35 after color/magnitude cuts: 80400
median:  -1.100000023841858
mean:  -1.1404619792506554
standard deviation:  0.674085393647737
number of initial stars in field 37: 1743130
number of  stars in field 37 after color/magnitude cuts: 296683
median:  -1.2070218324661255
mean:  -1.2409100695838062
standard deviation:  

/Users/amyel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in greater_equal
/Users/amyel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in less_equal


number of initial stars in field 51: 1574573
number of  stars in field 51 after color/magnitude cuts: 384681
median:  -1.2070218324661255
mean:  -1.2572755797732513
standard deviation:  0.644790291167799


In [105]:
print(field_number)
print(len(field_number))
print(initial_star_count)
print(len(initial_star_count))
print(cut_star_count)
print(len(cut_star_count))
print(mean_metallicity)
print(len(mean_metallicity))
print(median_metallicity)
print(len(median_metallicity))
print(std)
print(len(std))

[30, 32, 34, 35, 37, 40, 42, 46, 48, 49, 50, 51]
12
[1551563, 1825281, 1421909, 1606580, 1743130, 1752944, 1882163, 1304849, 2145968, 1763774, 1155082, 1574573]
12
[110478, 360148, 423023, 80400, 296683, 547078, 163259, 419384, 207284, 441649, 115484, 384681]
12
[-1.2790842923674013, -1.1976510338658024, -1.3951076149374833, -1.1404619792506554, -1.2409100695838062, -1.2267745602830837, -1.1385983824978068, -1.373064676084278, -1.2062539826482928, -1.1858527213223955, -1.401382761804954, -1.2572755797732513]
12
[-1.274411916732788, -1.1198506355285645, -1.447492003440857, -1.100000023841858, -1.2070218324661255, -1.1839725971221924, -1.100000023841858, -1.447492003440857, -1.1198506355285645, -1.1509276628494263, -1.3947120308876038, -1.2070218324661255]
12
[0.670759679741447, 0.6646917111479915, 0.6380655075651893, 0.674085393647737, 0.6619515582372671, 0.6460323769518082, 0.6697789494767419, 0.6837236484793757, 0.7047148182138855, 0.6135754051763209, 0.7561670702901315, 0.64479029116

In [110]:
print("Total number of initial LMC main body stars: "+str(sum(initial_star_count)))
print("Total number of LMC main body stars after color/magnitude cuts: "+str(sum(cut_star_count)))

Total number of initial LMC main body stars: 19727816
Total number of LMC main body stars after color/magnitude cuts: 3549551


### Make table for 12 SMASH LMC main body fields

In [106]:
arrays = [(np.array(field_number)),(np.array(initial_star_count)),(np.array(cut_star_count)),(np.array(mean_metallicity)).round(decimals=3),(np.array(median_metallicity)).round(decimals=3),(np.array(std)).round(decimals=3)]


In [107]:
inner_LMC_table = Table(arrays,names=('Field Number','Initial Star Count','Star Count after Color/Magnitude Cuts','Mean Metallicity','Median Metallicity','Standard Deviation'))


In [108]:
inner_LMC_table.pprint(max_lines=-1, max_width=-1)

Field Number Initial Star Count Star Count after Color/Magnitude Cuts Mean Metallicity Median Metallicity Standard Deviation
------------ ------------------ ------------------------------------- ---------------- ------------------ ------------------
          30            1551563                                110478           -1.279             -1.274              0.671
          32            1825281                                360148           -1.198              -1.12              0.665
          34            1421909                                423023           -1.395             -1.447              0.638
          35            1606580                                 80400            -1.14               -1.1              0.674
          37            1743130                                296683           -1.241             -1.207              0.662
          40            1752944                                547078           -1.227             -1.184              0.646


In [109]:
ascii.write(inner_LMC_table,'12 SMASH LMC Main Body Fields.csv',format='csv', fast_writer=False)

### Run metallicity function for SMC periphery fields

In [30]:
if __name__=="__main__":
    for number in SMC_outer_fields:
        metallicity('/Users/amyel/research/SMASHproject/datafiles/SMASH_fields/vsix/starsthree/Field{}_allobj_deep_stars.fits.gz'.format(number))

              

number of initial stars in field 1: 46634
number of  stars in field 1 after color/magnitude cuts: 4955
median:  -1.1839725971221924
mean:  -1.2195487196929753
standard deviation:  0.6819104555991227
number of initial stars in field 13: 80790
number of  stars in field 13 after color/magnitude cuts: 6015
median:  -1.447492003440857
mean:  -1.4123456668506786
standard deviation:  0.6527095165914374
number of initial stars in field 149: 69283
number of  stars in field 149 after color/magnitude cuts: 912
median:  -1.5315862894058228
mean:  -1.4715719839324064
standard deviation:  0.56929508573737
number of initial stars in field 150: 46060
number of  stars in field 150 after color/magnitude cuts: 564
median:  -1.5315862894058228
mean:  -1.488546660241995
standard deviation:  0.5731003838742754
number of initial stars in field 176: 111318
number of  stars in field 176 after color/magnitude cuts: 13328
median:  -1.447492003440857
mean:  -1.417238399752326
standard deviation:  0.63633178814863

In [31]:
print(field_number)
print(len(field_number))
print(initial_star_count)
print(len(initial_star_count))
print(cut_star_count)
print(len(cut_star_count))
print(mean_metallicity)
print(len(mean_metallicity))
print(median_metallicity)
print(len(median_metallicity))
print(std)
print(len(std))

[1, 13, 149, 150, 176, 177, 178, 18, 20, 21, 22, 8]
12
[46634, 80790, 69283, 46060, 111318, 98553, 233226, 77413, 42362, 46188, 59954, 53873]
12
[4955, 6015, 912, 564, 13328, 3515, 37355, 7460, 588, 817, 1038, 1070]
12
[-1.2195487196929753, -1.4123456668506786, -1.4715719839324064, -1.488546660241995, -1.417238399752326, -1.3444519109274573, -1.392215363763462, -1.4206483915390373, -1.7858262050896883, -1.429251514787051, -1.7849362181156005, -1.5152192796434252]
12
[-1.1839725971221924, -1.447492003440857, -1.5315862894058228, -1.5315862894058228, -1.447492003440857, -1.3419320583343506, -1.447492003440857, -1.447492003440857, -1.8289661407470703, -1.447492003440857, -1.8289661407470703, -1.5725317001342773]
12
[0.6819104555991227, 0.6527095165914374, 0.56929508573737, 0.5731003838742754, 0.6363317881486384, 0.6267477303683522, 0.6475469026567353, 0.6420830138247304, 0.5191283779353782, 0.6365739427900705, 0.5460905919883069, 0.605623313357807]
12


In [32]:
print("Total number of initial SMC peripnery stars: "+str(sum(initial_star_count)))
print("Total number of SMC periphery stars after color/magnitude cuts: "+str(sum(cut_star_count)))

Total number of initial SMC peripnery stars: 965654
Total number of SMC periphery stars after color/magnitude cuts: 77617


### Make table for 12 SMC periphery fields

In [12]:
arrays = [(np.array(field_number)),(np.array(initial_star_count)),(np.array(cut_star_count)),(np.array(mean_metallicity)).round(decimals=3),(np.array(median_metallicity)).round(decimals=3),(np.array(std)).round(decimals=3)]


In [14]:
outer_SMC_table = Table(arrays,names=('Field Number','Initial Star Count','Star Count after Color/Magnitude Cuts','Mean Metallicity','Median Metallicity','Standard Deviation'))


In [16]:
outer_SMC_table.pprint(max_lines=-1, max_width=-1)

Field Number Initial Star Count Star Count after Color/Magnitude Cuts Mean Metallicity Median Metallicity Standard Deviation
------------ ------------------ ------------------------------------- ---------------- ------------------ ------------------
           1              46634                                  4955            -1.22             -1.184              0.682
          13              80790                                  6015           -1.412             -1.447              0.653
         149              69283                                   912           -1.472             -1.532              0.569
         150              46060                                   564           -1.489             -1.532              0.573
         176             111318                                 13328           -1.417             -1.447              0.636
         177              98553                                  3515           -1.344             -1.342              0.627


In [20]:
ascii.write(outer_SMC_table,'12 SMASH SMC Periphery Fields.csv',format='csv', fast_writer=False)

### Run metallicity function for SMC main body fields

In [36]:
if __name__=="__main__":
    for number in SMC_inner_fields:
        metallicity('/Users/amyel/research/SMASHproject/datafiles/SMASH_fields/vsix/starsthree/Field{}_allobj_deep_stars.fits.gz'.format(number))

              

number of initial stars in field 12: 515707
number of  stars in field 12 after color/magnitude cuts: 87455
median:  -1.447492003440857
mean:  -1.4268286189194352
standard deviation:  0.6680220836845192
number of initial stars in field 14: 776768
number of  stars in field 14 after color/magnitude cuts: 112334
median:  -1.447492003440857
mean:  -1.4195000441249785
standard deviation:  0.6386960069279124
number of initial stars in field 15: 755871
number of  stars in field 15 after color/magnitude cuts: 112511
median:  -1.4035937786102295
mean:  -1.3553907278746795
standard deviation:  0.6602790958203677
number of initial stars in field 16: 402062
number of  stars in field 16 after color/magnitude cuts: 66450
median:  -1.447492003440857
mean:  -1.4270475017946493
standard deviation:  0.6539121848184676
number of initial stars in field 19: 43871
number of  stars in field 19 after color/magnitude cuts: 4176
median:  -1.447492003440857
mean:  -1.4016611789593956
standard deviation:  0.676141

In [37]:
print(field_number)
print(len(field_number))
print(initial_star_count)
print(len(initial_star_count))
print(cut_star_count)
print(len(cut_star_count))
print(mean_metallicity)
print(len(mean_metallicity))
print(median_metallicity)
print(len(median_metallicity))
print(std)
print(len(std))

[12, 14, 15, 16, 19, 2, 3, 4, 5, 7, 9]
11
[515707, 776768, 755871, 402062, 43871, 61991, 847797, 640420, 1048414, 1201514, 723837]
11
[87455, 112334, 112511, 66450, 4176, 6052, 139917, 120362, 194107, 239575, 106873]
11
[-1.4268286189194352, -1.4195000441249785, -1.3553907278746795, -1.4270475017946493, -1.4016611789593956, -1.3222157347387349, -1.3344073560374936, -1.3648531536862418, -1.44244677015881, -1.3519769477285566, -1.4272637965702424]
11
[-1.447492003440857, -1.447492003440857, -1.4035937786102295, -1.447492003440857, -1.447492003440857, -1.4035937786102295, -1.274411916732788, -1.4035937786102295, -1.5298746824264526, -1.4035937786102295, -1.447492003440857]
11
[0.6680220836845192, 0.6386960069279124, 0.6602790958203677, 0.6539121848184676, 0.6761414338047171, 0.6735859273558701, 0.6466056968765057, 0.6395829686207126, 0.6441452725148406, 0.6522756455044538, 0.6440043844245964]
11


In [38]:
print("Total number of initial SMC main body stars: "+str(sum(initial_star_count)))
print("Total number of SMC main body stars after color/magnitude cuts: "+str(sum(cut_star_count)))

Total number of initial SMC main body stars: 7018252
Total number of SMC main body stars after color/magnitude cuts: 1189812


### Make table for 11 SMC main body fields

In [39]:
arrays = [(np.array(field_number)),(np.array(initial_star_count)),(np.array(cut_star_count)),(np.array(mean_metallicity)).round(decimals=3),(np.array(median_metallicity)).round(decimals=3),(np.array(std)).round(decimals=3)]


In [40]:
inner_SMC_table = Table(arrays,names=('Field Number','Initial Star Count','Star Count after Color/Magnitude Cuts','Mean Metallicity','Median Metallicity','Standard Deviation'))


In [41]:
inner_SMC_table.pprint(max_lines=-1, max_width=-1)

Field Number Initial Star Count Star Count after Color/Magnitude Cuts Mean Metallicity Median Metallicity Standard Deviation
------------ ------------------ ------------------------------------- ---------------- ------------------ ------------------
          12             515707                                 87455           -1.427             -1.447              0.668
          14             776768                                112334            -1.42             -1.447              0.639
          15             755871                                112511           -1.355             -1.404               0.66
          16             402062                                 66450           -1.427             -1.447              0.654
          19              43871                                  4176           -1.402             -1.447              0.676
           2              61991                                  6052           -1.322             -1.404              0.674


In [42]:
ascii.write(outer_SMC_table,'11 SMASH SMC Main Body Fields.csv',format='csv', fast_writer=False)